In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


In [2]:
!pip install keras

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%%writefile app.py

import streamlit as st
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from keras.models import load_model
import io
from PIL import Image
import tempfile
import os

# Function to create and save Mel Spectrogram as an image
def create_mel_spectrogram(file_path, save_path, n_mels=128, hop_length=512):
    try:
        # Load the audio file
        y, sr = librosa.load(file_path, sr=None)

        # Create the Mel Spectrogram
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)
        S_DB = librosa.power_to_db(S, ref=np.max)

        # Plot and save the Mel Spectrogram
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Mel Spectrogram')
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    except Exception as e:
        st.error(f"Error creating mel spectrogram: {e}")

# Load the pre-trained model
model = load_model('/content/drive/MyDrive/Final Project DS3/vgg16_model_final1.h5')

# Function to make predictions
def make_prediction(image_path):
    try:
        # Load and preprocess the Mel Spectrogram image
        image = Image.open(image_path).convert('L')
        image = image.resize((224, 224))  # Resize to 224x224
        image = np.array(image)
        image = np.stack((image,) * 3, axis=-1)  # Convert to 3 channels
        image = image / 255.0  # Normalize pixel values
        image = image.reshape(1, 224, 224, 3)  # Reshape to match model input

        # Make predictions
        y_pred = model.predict(image)
        # For binary classification, you can use a threshold of 0.5
        y_pred_classes = (y_pred > 0.5).astype(int)
        return y_pred_classes
    except Exception as e:
        st.error(f"Error making prediction: {e}")
        return None

# Streamlit app
def main():
    st.title("Audio Classification")

    # File uploader widget
    uploaded_file = st.file_uploader("Upload Audio", type=["wav", "mp3", "flac"])

    if uploaded_file:
        # Save the uploaded file temporarily
        try:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp_file:
                audio_file_path = tmp_file.name
                tmp_file.write(uploaded_file.getbuffer())

            # Create and save Mel Spectrogram image
            mel_spectrogram_path = "mel_spectrogram.png"
            create_mel_spectrogram(audio_file_path, mel_spectrogram_path)

            # Perform prediction
            prediction = make_prediction(mel_spectrogram_path)
            if prediction is not None:
                # Display prediction
                st.write(f"Predicted class: {'Real' if prediction[0] == 1 else 'Fake'}")

                # Display the Mel Spectrogram
                image = Image.open(mel_spectrogram_path)
                st.image(image, caption='Mel Spectrogram', use_column_width=True)
        except Exception as e:
            st.error(f"Error processing uploaded file: {e}")
        finally:
            if os.path.exists(audio_file_path):
                os.remove(audio_file_path)

if __name__ == "__main__":
    main()


Overwriting app.py


In [2]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 2.844s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [3]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.106.119.117


In [4]:
!streamlit run app.py &>/content/logs.txt &

In [5]:
!npx localtunnel --port 8501

npx: installed 22 in 6.676s
your url is: https://wicked-keys-fix.loca.lt
^C
